In [1]:
from dotenv import load_dotenv
import os
from apikeys_GEMINI import APIKeyManager

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

In [10]:
load_dotenv(dotenv_path=r'.env')

MODEL_GEMINI = os.getenv("MODEL_GEMIMI")
# MODEL_GEMINI = "models/gemini-1.5-pro"
if MODEL_GEMINI is None:
    raise ValueError("Environment variable MODEL_GEMINI is not set")
elif not MODEL_GEMINI.startswith("models/"):
    MODEL_GEMINI = f"models/{MODEL_GEMINI}"


URL_QDRANT = os.getenv("URL_QDRANT")
API_QDRANT = os.getenv("API_QDRANT")
CONTENT_PAYLOAD_KEY = os.getenv("CONTENT_PAYLOAD_KEY")
METADATA_PAYLOAD_KEY = os.getenv("METADATA_PAYLOAD_KEY")

API_GEMINI_LIST = os.getenv('APIS_GEMINI').split(',')
key_manager = APIKeyManager(API_GEMINI_LIST)

TOP_K = 5
MAX_DOCS_FOR_CONTEXT = 8

# Hàm sinh câu hỏi tương tự (3 câu)

In [14]:
def query_generator(original_query: str) -> list[str]:
    """Generate queries from original query"""
    # Câu truy vấn gốc
    query = original_query
    
    # Cập nhật prompt để yêu cầu rõ ràng chỉ trả về 3 câu truy vấn và câu gốc
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant that generates multiple search queries based on a single query."),
            ("human", "Generate 3 search queries related to: {original_query}. Provide each query on a new line, and ensure that only the queries are returned, with no additional explanations or text."),
        ]
    )
    
    model = ChatGoogleGenerativeAI(
        # key api google gemini, nếu test mà bị báo lỗi api core thì lấy api khác trong .env để test
        google_api_key=key_manager.get_next_key(),
        model=MODEL_GEMINI,
        temperature=0
    )
    
    query_generator_chain = (
        prompt | model | StrOutputParser()
    )
    
    # Kết quả sẽ là một chuỗi các câu truy vấn cách nhau bằng dấu xuống dòng
    result = query_generator_chain.invoke({"original_query": query})
    
    # Tách kết quả thành danh sách các câu truy vấn
    generated_queries = result.strip().split('\n')
    
    # Đảm bảo chỉ lấy 3 câu nếu có nhiều hơn 3 câu truy vấn sinh ra
    if len(generated_queries) > 3:
        generated_queries = generated_queries[:3]
    
    # Kết hợp câu gốc với các câu truy vấn sinh ra
    queries = [query] + generated_queries
    
    return queries


# Test

In [15]:
# Dictionary để test hàm query_generator
test_input = "Tôi tốt nghiệp cao đẳng dược, đã có chứng chỉ hành nghề y và làm việc tại 1 bệnh viện thuộc TP. Hải Phòng. Xin hỏi, tôi muốn làm thêm chứng chỉ hành nghề dược có được không? Có phải bỏ chứng chỉ y để làm chứng chỉ dược không? Có thể sử dụng song song 2 chứng chỉ này được không?"

# Gọi hàm query_generator với test_input
generated_queries = query_generator(test_input)

# In ra các truy vấn được sinh ra
for i, query in enumerate(generated_queries, 1):
    print(f"{i}: {query}")


1: Tôi tốt nghiệp cao đẳng dược, đã có chứng chỉ hành nghề y và làm việc tại 1 bệnh viện thuộc TP. Hải Phòng. Xin hỏi, tôi muốn làm thêm chứng chỉ hành nghề dược có được không? Có phải bỏ chứng chỉ y để làm chứng chỉ dược không? Có thể sử dụng song song 2 chứng chỉ này được không?
2: Điều kiện làm thêm chứng chỉ hành nghề dược khi đã có chứng chỉ hành nghề y
3: Có thể sử dụng đồng thời chứng chỉ hành nghề y và chứng chỉ hành nghề dược
4: Bỏ chứng chỉ hành nghề y để làm chứng chỉ hành nghề dược


In [16]:
generated_queries[1]

'Điều kiện làm thêm chứng chỉ hành nghề dược khi đã có chứng chỉ hành nghề y'

In [17]:
type(generated_queries[1])

str